In [9]:
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import h5py
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image

In [ ]:
img = np.array(Image.open("content/datasets/data_in/objects/sim/segm/0018/clone/00000.png"))
obj_ids = np.unique(img)
# to correctly interpret the id of a single object
print(obj_ids)
obj_id = obj_ids[0]
class_id = obj_id // 1000
print(class_id)
obj_instance_id = obj_id % 1000
print(obj_instance_id)

In [11]:


unique=0

def interpolate_color(index,unique):

    # Calculate the RGB components using linear interpolation
    r = int(index * (255 / unique))
    g = int((unique - index) * (255 / unique))
    b = 128  # Set a constant value for blue (change this value for a different shade of blue)

    return r, g, b




def map_to_r(number):
    colorr = [255, 128, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 255, 128, 255, 0, 128]
    if number<20:
      return int(colorr[number])
    else:
      return 0

def map_to_g(number):

    colorg = [0, 128, 255, 128, 0, 0, 128, 0, 0, 128, 0, 128, 0, 128, 255, 128, 0, 128, 255, 128]

    if number<20:
      return int(colorg[number])
    else:
      return 0

def map_to_b(number):

    colorb = [0, 0, 0, 128, 128, 255, 255, 128, 0, 128, 128, 255, 128, 0, 0, 0, 128, 255, 255, 128]

    if number<20:
      return int(colorb[number])
    else:
      return 255



In [12]:


def parse_txt_file(txt_file):
    color_to_class_id = {}
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:  # Skip the header line
            parts = line.strip().split()
            class_name = parts[0]
            # If the class_name starts with "Car:", change it to just "Car"
            rgb_values = tuple(map(int, parts[1:]))
            color_to_class_id[rgb_values] = class_name
    return color_to_class_id

def rgb_mask_to_class_ids(rgb_mask, class_to_id, label_id_mapping):

    # Create a lookup table for faster mapping
    lookup_table = np.zeros((256, 256, 256), dtype=np.uint8)
    for rgb_pixel, class_name in class_to_id.items():
        if class_name.startswith("Car") or class_name.startswith("Van"):
            class_name = "Car"
        lookup_table[rgb_pixel] = label_id_mapping[class_name]

    # Convert the RGB mask to a 1D array of unique colors
    flat_rgb_mask = rgb_mask.reshape(-1, 3)

    # Use the lookup table to map the colors to class IDs
    class_ids = lookup_table[flat_rgb_mask[:, 0], flat_rgb_mask[:, 1], flat_rgb_mask[:, 2]]

    # Reshape the class IDs back to the original image shape
    class_ids = class_ids.reshape(rgb_mask.shape[:2])

    return class_ids




def interpolate_color(index, unique):
    r = int(index * (255 / unique))
    g = int((unique - index) * (255 / unique))
    b = 128  # Set a constant value for blue (change this value for a different shade of blue)
    return r, g, b

In [15]:
def load_sim_data(dataset_index,perturbation,intensity,stop=0):
  images_list_sim = []
  bounding_boxes_list_sim = []
  images_list_index_sim = []
  semantic_id_list_sim = []
  raw_image_sim = []

  label_id_mapping = {
        'Road': 0,
        'Sidewalk': 1,
        'Building': 2,
        'Wall': 2,
        'Fence': 2,
        'Pole': 2,
        'TrafficLight': 6,
        'TrafficSign': 7,
        'Vegetation': 8,
        'Tree': 8,
        'Terrain': 1,
        'Sky': 10,
        'Person': 11,
        'Rider': 12,
        'Car': 13,
        'Truck': 14,
        'Bus': 15,
        'Train': 16,
        'Motorcycle': 17,
        'Bicycle': 18,
        'Void': 255,
        'Misc': 2,
        'GuardRail': 21,
    }





  # Update the paths to the dataset folders accordingly
  images_folder = f"./content/datasets/data_in/objects/sim_perturbed/img/{dataset_index}/clone/{perturbation}_{intensity}"
  labels_file = f"./content/datasets/data_in/objects/sim/obj/{dataset_index}_clone.txt"
  segmentation_masks_folder = f"./content/datasets/data_in/objects/sim/segm/{dataset_index}/clone"
  txt_file_path = f'./content/datasets/data_in/encodings/{dataset_index}_clone_scenegt_rgb_encoding.txt'
  color_to_class_id_mapping = parse_txt_file(txt_file_path)

  # Read the ground truth labels from the text file
  with open(labels_file, 'r') as f:
      lines = f.readlines()[1:]  # Skip the header line

  # Dictionary to store bounding boxes or labels for each image index
  ground_truth_data = {}

  image_files = sorted(os.listdir(images_folder))
  max_index=int(image_files[-1].strip().split('.png')[0])
  for index in range(0,max_index+1):
    if index not in ground_truth_data:
                  ground_truth_data[index] = []
  # Process ground truth lines and group them by image index
  for line in lines:
      line_parts = line.strip().split(' ')
      image_index = int(line_parts[0])

      if(image_index<=stop or stop==0):
          label = line_parts[2]

          if label != 'DontCare':
              x_min = int(float(line_parts[6]))
              y_min = int(float(line_parts[7]))
              x_max = int(float(line_parts[8]))
              y_max = int(float(line_parts[9]))

              # Save bounding box or label in the dictionary

              ground_truth_data[image_index].append((label, x_min, y_min, x_max, y_max))

  # Process images in the sorted order
  for image_index in sorted(ground_truth_data.keys()):
      if(image_index%50==0):
        print(image_index)
      image_file = sorted(os.listdir(images_folder))[image_index]
      image_path = os.path.join(images_folder, image_file)
      image = cv2.imread(image_path)
      raw_image_sim.append(image)



      bounding_boxes = ground_truth_data[image_index]
      image2 = image.copy()

      for label, x_min, y_min, x_max, y_max in bounding_boxes:
          if x_max > x_min and y_max > y_min:
              cv2.rectangle(image2, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
              cv2.putText(image2, label, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              bounding_boxes_list_sim.append((image_index,label, x_min, y_min, x_max, y_max))


      segmentation_mask_file = f"{os.path.splitext(image_file)[0]}.png"
      segmentation_mask_path = os.path.join(segmentation_masks_folder, segmentation_mask_file)
      if os.path.exists(segmentation_mask_path):
          segmentation_mask = cv2.imread(segmentation_mask_path)

          images_list_index_sim.append(image_index)
          images_list_sim.append(image2)
          semantic_id = rgb_mask_to_class_ids(segmentation_mask[:,:,::-1], color_to_class_id_mapping,label_id_mapping)
          semantic_id_list_sim.append(semantic_id)

  return semantic_id_list_sim,raw_image_sim

In [16]:
raw_image_sim = {}
semantic_id_list_sim = {}
dataset_index_list=["0018"]
functions = ['canny_edges_mapping', 'contrast','defocus_blur','dotted_lines_mapping',   
             'fog_filter', 'frost_filter', 'gaussian_blur', 'gaussian_noise', 
             'glass_blur', 'grayscale_filter', 'histogram_equalisation','impulse_noise', 'increase_brightness', 'jpeg_filter', 
             'low_pass_filter', 'motion_blur', 'object_overlay',  
             'phase_scrambling', 'pixelate', 'poisson_noise','posterize_filter','saturation_filter', 'sharpen_filter']

for dataset_index in dataset_index_list:
  for perturbation in functions:
    for intensity in range(4,5):
      semantic_id_list_sim_out,raw_image_sim_out=load_sim_data(dataset_index,perturbation,intensity)
      raw_image_sim[dataset_index+"_"+perturbation+"_"+str(intensity)] = raw_image_sim_out
      # semantic_id_list_sim[dataset_index+"_"+perturbation+"_"+str(intensity)] = semantic_id_list_sim_out
      print(len(raw_image_sim[dataset_index+"_"+perturbation+"_"+str(intensity)]))
      print("-DONE- ", dataset_index, perturbation, intensity)

0
50
100
150
200
250
300
339
-DONE-  0018 canny_edges_mapping 4
0
50
100
150
200
250
300
339
-DONE-  0018 contrast 4
0
50
100
150
200
250
300
339
-DONE-  0018 defocus_blur 4
0
50
100
150
200
250
300
339
-DONE-  0018 dotted_lines_mapping 4
0
50
100
150
200
250
300
339
-DONE-  0018 fog_filter 4
0
50
100
150
200
250
300
339
-DONE-  0018 frost_filter 4
0
50
100
150
200
250
300
339
-DONE-  0018 gaussian_blur 4
0
50
100
150
200
250
300
339
-DONE-  0018 gaussian_noise 4
0
50
100
150
200
250
300
339
-DONE-  0018 glass_blur 4
0
50
100
150
200
250
300
339
-DONE-  0018 grayscale_filter 4
0
50
100
150
200
250
300
339
-DONE-  0018 histogram_equalisation 4
0
50
100
150
200
250
300
339
-DONE-  0018 impulse_noise 4
0
50
100
150
200
250
300
339
-DONE-  0018 increase_brightness 4
0
50
100
150
200
250
300
339
-DONE-  0018 jpeg_filter 4
0
50
100
150
200
250
300
339
-DONE-  0018 low_pass_filter 4
0
50
100
150
200
250
300
339
-DONE-  0018 motion_blur 4
0
50
100
150
200
250
300
339
-DONE-  0018 object_overla

In [ ]:
!mkdir content/datasets/h5_out_perturbed/

def save_dictionary_to_h5(file_path, data_dict):
    with h5py.File(file_path, 'w') as hf:
        for key, array_list in data_dict.items():
            # Create a group for each key in the dictionary
            group = hf.create_group(key)
            for i, arr in enumerate(array_list):
                # Save each array as a dataset within the group
                group.create_dataset(f'array_{i}', data=arr)



file_path = 'content/datasets/h5_out_perturbed/raw_image_sim.h5'
save_dictionary_to_h5(file_path, raw_image_sim)


file_path = 'content/datasets/h5_out_perturbed/semantic_id_list_sim.h5'
save_dictionary_to_h5(file_path, semantic_id_list_sim)
